In [2]:
#Imports
import pandas as pd
import functions as func
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import scipy.stats as stats

%matplotlib inline

db_locations = func.import_yaml()

#load df
df = pd.read_csv(db_locations['data_clean']['user_logs_var'])

In [2]:
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,prev_time,time_diff,time_diff_seconds,error,Variation
0,6265388,100019786_4272121951,703380671_89376183829_330646,start,2017-04-15 15:13:33,NaN,NaN,NaN,0,Test
1,6265388,100019786_4272121951,703380671_89376183829_330646,step_1,2017-04-15 15:13:46,2017-04-15 15:13:33,0 days 00:00:13,13.0,0,Test
2,6265388,100019786_4272121951,703380671_89376183829_330646,step_2,2017-04-15 15:14:21,2017-04-15 15:13:46,0 days 00:00:35,35.0,0,Test
3,6265388,100019786_4272121951,703380671_89376183829_330646,step_3,2017-04-15 15:15:50,2017-04-15 15:14:21,0 days 00:01:29,89.0,0,Test
4,6265388,100019786_4272121951,703380671_89376183829_330646,confirm,2017-04-15 15:16:18,2017-04-15 15:15:50,0 days 00:00:28,28.0,0,Test


In [3]:
#Hypothesis Testing for Completion Rate
#Null Hypothesis:  There is no significant difference in completion rates
#Alternative Hypothesis:  The redesign has led to a significant change
#Treshold = 0.05
#by client

z_stat, p_value = func.completion_rate_hypothesis_test(df,"client_id")
print(f"z-Statistic: {z_stat}, p-Value: {p_value}")

p_value < 0.05

Grouping by client_id
Successes (Control, Test): [15429, 18681]
Totals (Control, Test): [23527, 26960]
z-Statistic: -8.886541033708866, p-Value: 6.304077732636919e-19


True

In [4]:
#By visit_id

z_stat, p_value = func.completion_rate_hypothesis_test(df,"visit_id")
print(f"z-Statistic: {z_stat}, p-Value: {p_value}")

p_value < 0.05

Grouping by visit_id
Successes (Control, Test): [16040, 21714]
Totals (Control, Test): [32183, 37109]
z-Statistic: -22.867662620559358, p-Value: 9.751835523847512e-116


True

In [20]:
#Hypothesis Testing for Completion Rate with a 5% increase
#Null Hypothesis:  There is no significant difference in completion rates
#Alternative Hypothesis:  The redesign has led to a significant change
#Treshold = 0.05
#by visit_id



# Calculate completion counts (completing when reaching the 'confirm' step)
completion_counts = df[df['process_step'] == 'confirm'].groupby('Variation').size()

# Total number of participants in each group
total_counts = df.groupby('Variation')['visit_id'].nunique()

# Completion rate for each group
completion_rates = completion_counts / total_counts

# Define the cost-effectiveness threshold (+5% to Control group completion rate)
control_rate = completion_rates['Control']
threshold = control_rate + 0.05

# Completion rate for Test group
test_rate = completion_rates['Test']

# Perform two-proportion z-test
# n_control: Total participants in Control group, n_test: Total participants in Test group
n_control = total_counts['Control']
n_test = total_counts['Test']

# x_control: Completions in Control group, x_test: Completions in Test group
x_control = completion_counts['Control']
x_test = completion_counts['Test']

# Pooled proportion
p_pooled = (x_control + x_test) / (n_control + n_test)

# Standard error
se = np.sqrt(p_pooled * (1 - p_pooled) * (1/n_control + 1/n_test))

# Z-statistic (for one-sided test)
z_stat = (test_rate - threshold) / se

# P-value for one-sided test
p_value = 1 - stats.norm.cdf(z_stat)

# Significance level
alpha = 0.05

# Output
print(f"Control Completion Rate: {control_rate:.3f}")
print(f"Test Completion Rate: {test_rate:.3f}")
print(f"Threshold (Control Rate + 5%): {threshold:.3f}")
print(f"Z-statistic: {z_stat:.3f}")
print(f"P-value: {p_value:.3f}")

# Hypothesis testing decision
if p_value < alpha:
    print("Reject the null hypothesis: Test group's completion rate is significantly greater than the threshold.")
else:
    print("Fail to reject the null hypothesis: Test group's completion rate is not significantly greater than the threshold.")

Control Completion Rate: 0.538
Test Completion Rate: 0.689
Threshold (Control Rate + 5%): 0.588
Z-statistic: 27.264
P-value: 0.000
Reject the null hypothesis: Test group's completion rate is significantly greater than the threshold.


In [5]:
#Hypothesis Testing for Error Rate
#Null Hypothesis:  There is no significant difference in error rates
#Alternative Hypothesis:  The redesign has led to a significant change
#by client id

In [5]:
z_stat, p_value = func.error_rate_hypothesis_test(df, "client_id")
print(f"z-Statistic: {z_stat}, p-Value: {p_value}")

p_value < 0.05

Grouping by client_id
Errors (Control, Test): [2703, 3222]
Totals (Control, Test): [23527, 26960]
z-Statistic: -1.6093583502630044, p-Value: 0.10753800691954607


False

In [6]:
#By visit_id
z_stat, p_value = func.error_rate_hypothesis_test(df,"visit_id")
print(f"z-Statistic: {z_stat}, p-Value: {p_value}")

p_value < 0.05

Grouping by visit_id
Errors (Control, Test): [2774, 3441]
Totals (Control, Test): [32183, 37109]
z-Statistic: -3.00124616129456, p-Value: 0.002688771091388191


True

In [8]:
#Hypothesis Testing for completion time  Rate
#Null Hypothesis:  There is no significant difference in completion rates
#Alternative Hypothesis:  The redesign has led to a significant change



In [3]:
z_stat, p_value = func.time_complete_hypothesis_test(df,"client_id")
print(f"z-Statistic: {z_stat}, p-Value: {p_value}")
p_value < 0.05

z-Statistic: 0.6836840469172146, p-Value: 0.494175199747517


False

In [4]:
#z_stat, p_value = func.time_complete_hypothesis_test(df,"visit_id")
print(f"z-Statistic: {z_stat}, p-Value: {p_value}")


TypeError: unsupported operand type(s) for /: 'str' and 'int'

False